Fit an LDA topic model to full dataset of Capital Punishment Discussions and visualize the result

Hunter Priniski

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [3]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['gun', 'control', 'firearm', 're'])

In [11]:
#For those who are running this off of GitHub, use: data/full_df.xlsx
data = pd.read_excel('data.xlsx')

In [12]:
#prepare text
def prepare(textcol, unique = False):
    
    texts = textcol.tolist()  
    
    if unique == True: 
        texts = set(texts)
    
    prepared_text = []
    
    for text in texts:
        prepared_text.append(gensim.utils.simple_preprocess(str(text), deacc=True))
    
    return prepared_text

In [13]:
#pass prepare a pd column of text data
prepared = prepare(data['selftext'], unique = True)

In [14]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(prepared, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[prepared], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [15]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [16]:
# Remove Stop Words
data_words_nostops = remove_stopwords(prepared)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [17]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
#these are the vectors that we will change by training on a word2Vec
corpus = [id2word.doc2bow(text) for text in texts]

In [19]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=150,
                                           passes=15,
                                           alpha='auto',
                                           per_word_topics=True)

In [20]:
pprint(lda_model.print_topics())

[(0,
  '0.019*"gun" + 0.014*"law" + 0.012*"people" + 0.012*"would" + 0.006*"right" '
  '+ 0.006*"want" + 0.006*"think" + 0.006*"believe" + 0.005*"country" + '
  '0.005*"work"'),
 (1,
  '0.012*"people" + 0.009*"would" + 0.009*"use" + 0.008*"gun" + 0.006*"person" '
  '+ 0.006*"also" + 0.006*"make" + 0.006*"shooting" + 0.005*"ownership" + '
  '0.005*"allow"'),
 (2,
  '0.020*"gun" + 0.015*"would" + 0.014*"people" + 0.012*"use" + 0.009*"ban" + '
  '0.008*"violence" + 0.007*"country" + 0.007*"believe" + 0.006*"reason" + '
  '0.006*"death"'),
 (3,
  '0.027*"http_www" + 0.016*"downvote" + 0.016*"rule" + 0.011*"people" + '
  '0.010*"would" + 0.009*"rifle" + 0.009*"effective" + 0.008*"report" + '
  '0.008*"guidelines_wiki_upvote" + 0.008*"change_view"'),
 (4,
  '0.027*"gun" + 0.018*"people" + 0.011*"right" + 0.009*"would" + '
  '0.009*"http_www" + 0.008*"law" + 0.007*"ban" + 0.007*"think" + 0.007*"year" '
  '+ 0.007*"want"'),
 (5,
  '0.014*"ammunition" + 0.010*"private" + 0.009*"firearm" + 0.008

In [21]:
doc_lda = lda_model[corpus]

In [22]:
# Compute Perplexity. this is a terrible measure, but for piloting purposes like this one, it will do. 
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.465367862704552

Coherence Score:  0.3188600322491709


In [23]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.020467  0.065202       1        1  21.488752
7      0.030786  0.019170       2        1  16.058208
0     -0.028403  0.080818       3        1  10.886990
4      0.021970  0.035776       4        1  10.786202
3      0.117215 -0.074691       5        1  10.685871
1     -0.010101  0.075414       6        1   9.339933
6      0.060621 -0.031853       7        1   6.623751
8      0.062201 -0.058950       8        1   6.295233
5     -0.168367 -0.098243       9        1   4.316714
9     -0.065455 -0.012642      10        1   3.518353, topic_info=          Term        Freq       Total Category  logprob  loglift
41    http_www  132.000000  132.000000  Default  30.0000  30.0000
34         gun  273.000000  273.000000  Default  29.0000  29.0000
76        rule   85.000000   85.000000  Default  28.0000  28.0000
246     people  255.000000  255.000000  Default  27.0000  27.0000
19    downvote   78.000000   78.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
1169    reduce    2.353600   38.961582  Topic10  -5.7040   0.5405
369      think    2.491520  109.068001  Topic10  -5.6470  -0.4319
302        ban    2.379697  108.462189  Topic10  -5.6929  -0.4723
524       seem    2.109432   47.707806  Topic10  -5.8135   0.2285
123       even    2.106047   78.245567  Topic10  -5.8151  -0.2679

[774 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
294       1  0.276422     able
294       2  0.307135     able
294       3  0.061427     able
294       5  0.122854     able
294       6  0.061427     able
...     ...       ...      ...
96        9  0.019332     year
1211      2  0.211832  youtube
1211      5  0.211832  youtube
1211      8  0.211832  youtube
1211     10  0.211832  youtube

[1989 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 8, 1, 5, 4, 2, 7, 9, 6, 10])